In [1]:
import deepchem as dc
import tensorflow as tf
import numpy as np
import rdkit
from rdkit import *

/home/cbilodeau/anaconda2/envs/deepchem2/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Chapter 9

In [8]:
tasks,datasets,transformers = dc.molnet.load_muv()
train_dataset,valid_dataset,test_dataset = datasets
train_smiles = train_dataset.ids

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [9]:
tokens = set()
for s in train_smiles: 
    tokens = tokens.union(set(s))
tokens = sorted(list(tokens))
max_length = max(len(s) for s in train_smiles)

In [15]:
from deepchem.models.tensorgraph.optimizers import Adam, ExponentialDecay
from deepchem.models.tensorgraph.models.seqtoseq import AspuruGuzikAutoEncoder
model = AspuruGuzikAutoEncoder(tokens,max_length,model_dir = 'vae')
batches_per_epoch = len(train_smiles)/model.batch_size
learning_rate = ExponentialDecay(0.001,0.95,batches_per_epoch)
model.set_optimizer(Adam(learning_rate=learning_rate))

In [ ]:
def generate_sequences(epochs):
    for i in range(epochs):
        for s in train_smiles:
            yield (s,s)
            
model.fit_sequences(generate_sequences(50))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [ ]:
#Generate list of molecules that follows SMILES string rules:
import numpy as np
from rdkit import Chem
predictions = model.predict_from_embeddings(np.random.normal(size=(100,196)))
molecules = []
for p in predictions:
    smiles = ''.join(p)
    if Chem.MolFromSmiles(smiles) is not None:
        molecules.append(smiles)
    for m in molecules:
        print(m)